# Initialization

In [22]:
!pip install -U PyYAML
import yaml
print("This should be > 5.0 ", yaml.__version__)
!git clone https://github.com/rahulanand16nov/Samsung-PRISM
!cp -r Samsung-PRISM/* .
!rm Model.ipynb*
!rm -rf Samsung-PRISM

This should be > 5.0  5.4.1
Cloning into 'Samsung-PRISM'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 50 (delta 9), reused 37 (delta 3), pack-reused 0
Unpacking objects: 100% (50/50), done.


# Download dataset and unzip

In [2]:
from pathlib import Path
my_file = Path("/content/CelebAMask-HQ.zip")

if not my_file.is_file():
  !gdown --id 1cDM_RMNrK9CeCn6qmDilu6u-AP2A9Qzs

!unzip "/content/CelebAMask-HQ.zip" -d "/content/datasets/celeba-hq/"
!rm /content/CelebAMask-HQ.zip



Streaming output truncated to the last 5000 lines.
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19597_neck.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19597_nose.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19597_r_brow.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19597_r_eye.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19597_skin.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19597_u_lip.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19598_hair.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19598_l_brow.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/CelebAMask-HQ-mask-anno/9/19598_l_eye.png  
  inflating: /content/datasets/celeba-hq/CelebAMask-HQ/Celeb

# Create image list

In [63]:
import os
import argparse
import numpy as np


parser = argparse.ArgumentParser()
parser.add_argument('--path', type=str, default='/content/test_set/data/')
parser.add_argument('--output', type=str, default='/content/datasets/celeba-hq/list_val.txt')
args = parser.parse_args(args=[])

ext = {'.jpg', '.png', '.JPG'}

images = []
for root, dirs, files in os.walk(args.path):
    print('loading ' + root)
    for file in files:
        if os.path.splitext(file)[1] in ext:
            images.append(os.path.join(root, file))
images = sorted(images)
np.savetxt(args.output, images, fmt='%s')

loading /content/test_set/data/


# Create mask list

In [62]:
!cp ./datasets/celeba-hq/CelebAMask-HQ/CelebA-HQ-img/1*.jpg ./test_set/data/

^C


In [ ]:
import cv2
import dlib
import os
import imutils
from PIL import Image

parser = argparse.ArgumentParser()
parser.add_argument('--path', type=str, default='/content/test_set/data/')
parser.add_argument('--output', type=str, default='datasets/mask/list_celeba-hq-irregular.txt')
args = parser.parse_args(args=[])

ext = {'.jpg', '.png', '.JPG'}

image_masks = []
for root, dirs, files in os.walk(args.path):
    print('loading ' + root)
    for file in files:
      # generate mask image
      if os.path.splitext(file)[1] in ext:
        print(os.path.join(root, file))
        img = cv2.imread(os.path.join(root, file))

        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        detector = dlib.get_frontal_face_detector()
        faces = detector(gray, 1)

        p = "shape_predictor_68_face_landmarks.dat"
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor(p)

        for face in faces:
          landmarks = predictor(gray, face)

        points = []
        for i in range(1, 16):
            point = [landmarks.part(i).x, landmarks.part(i).y]
            points.append(point)

        # Coordinates for the additional 3 points for wide, high coverage mask - in sequence
        mask = points +  [((landmarks.part(42).x), (landmarks.part(15).y)),
                  ((landmarks.part(27).x), (landmarks.part(27).y)),
                  ((landmarks.part(39).x), (landmarks.part(1).y))]

        fmask = np.array(mask, dtype=np.int32)

        # change parameter [mask_type] and color_type for various combination
        blank_image = np.zeros((img.shape[0],img.shape[1], 1), np.int32)
        img_with_outline = cv2.polylines(blank_image, [fmask], True, (255,255,255), thickness=2, lineType=cv2.LINE_8)

        # Using Python OpenCV – cv2.fillPoly() method to fill mask
        # change parameter [mask_type] and color_type for various combination
        img_mask_filled = cv2.fillPoly(img_with_outline, [fmask], (255,255,255), lineType=cv2.LINE_AA)
            #images.append(os.path.join(root, file))
        cv2.imwrite(os.path.join('/content/test_set/mask/', file), img_mask_filled)

        images.append(os.path.join('/content/test_set/mask/', file))

images = sorted(images)
np.savetxt(args.output, images, fmt='%s')

In [72]:
import argparse
import os
from utils import get_config, _write_images
import torch
from data import create_dataset, create_dataloader
from models.networks import define_G
from data.util import tensor2img
import skimage.io as sio
import numpy as np

parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='configs/celeba-hq-regular_list.yaml', help="net configuration")
parser.add_argument('--output_folder', type=str, default='outputs/celebahq-regular/saved_images', help="output image path")
parser.add_argument('--checkpoint', type=str, default='outputs/celebahq-regular/checkpoints/latest_G.pth',
                    help="checkpoint of generator")
opts = parser.parse_args(args=[])

if not os.path.exists(opts.output_folder):
    os.makedirs(opts.output_folder)

# Load experiment setting
config = get_config(opts.config)

device = torch.device('cuda')
# Setup model and data loader


model = define_G(config).to(device)
model.load_state_dict(torch.load(opts.checkpoint), strict=True)
model.eval()

print('Loading the checkpoint for G [{:s}] ...'.format(opts.checkpoint))

with torch.no_grad():
    dataset_opt = config['datasets']['test']
    test_set = create_dataset(dataset_opt)
    test_loader = create_dataloader(test_set, dataset_opt)
    print('Number of test images in [{:s}]: {:d}'.format(dataset_opt['name'], len(test_set)))

    # Start testing

    for index, test_data in enumerate(test_loader):
        v_input, v_output, v_target = [], [], []
        visual_images = []
        var_input, var_mask, var_target, img_paths = test_data['input'], test_data['mask'], test_data['target'], \
                                                     test_data['paths']
        var_input = var_input.to(device)
        var_mask = var_mask.to(device)
        var_target = var_target.to(device)
        var_output = var_mask.detach() * model(torch.cat([var_input, var_mask], dim=1)) + (
                1 - var_mask.detach()) * var_input.detach()
        v_input.append(var_input.detach()[0].float().cpu())
        v_output.append(var_output.detach()[0].float().cpu())
        v_target.append(var_target.detach()[0].float().cpu())
        visual_images.extend(v_input)
        visual_images.extend(v_output)
        visual_images.extend(v_target)
        _write_images(visual_images, 1, '%s/%s' % (opts.output_folder, img_paths[0].split('/')[-1]))
        saved_mask = (var_mask.detach()[0].float().cpu().numpy().squeeze() * 255).round().astype(np.uint8)
        saved_input = (var_mask.detach()[0].float().cpu() + ((v_target[0] + 1) / 2)).numpy().squeeze().transpose(1, 2, 0).clip(0, 1)
        saved_output = tensor2img(v_output)
        saved_target = tensor2img(v_target)
        sio.imsave(os.path.join(opts.output_folder, 'mask', img_paths[0].split('/')[-1].split('.')[0] + '.png'), saved_mask)
        sio.imsave(os.path.join(opts.output_folder, 'input', img_paths[0].split('/')[-1]), saved_input)
        sio.imsave(os.path.join(opts.output_folder, 'output', img_paths[0].split('/')[-1]), saved_output[0])
        sio.imsave(os.path.join(opts.output_folder, 'target', img_paths[0].split('/')[-1]), saved_target[0])


print('End of testing.')


Loading the checkpoint for G [outputs/celebahq-regular/checkpoints/latest_G.pth] ...
Number of test images in [celeba-hq]: 2259


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: outputs/celebahq-regular/saved_images/mask/10027.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: outputs/celebahq-regular/saved_images/input/10027.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: outputs/celebahq-regular/saved_images/mask/10028.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: outputs/celebahq-regular/saved_images/input/10028.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: outputs/celebahq-regular/saved_images/mask/10029.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: outputs/celebahq-regular/saved_images/input/10029.jpg is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: Us

KeyboardInterrupt: ignored